In [1]:
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import pandas as pd

Esto es para conectarnos a la base de datos

In [17]:
try:
    conexion= psycopg2.connect(
        database= "Chinook",
        user= "postgres",
        password= "admin",
        host= "localhost",
        port= "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es erronea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrió el error {e}")


El cursor es el tren que lleva la query(consulta) a SQL y quien trae la info a Python  
IMPORTANTE que una tiene que tener simples y otras dobles

In [18]:
cursor= conexion.cursor()
cursor.execute('SELECT * FROM "Genre";')         #Esto ejecuta la query en SQL (DBeaver)
df_genero= pd.DataFrame(cursor.fetchall())        #fetchall (fetchone es para traer solo uno) es lo que trae la ejecución a Python   nos da una lista de tuplas por eso hacemos un dataframe

Para cerrar la conexion 

In [19]:
cursor.close()
conexion.close()

Ya podríamos trabajar con lo que nos hemos traido

In [20]:
df_genero.head()

,0,1
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll


Vamos a conectar a otra y vamos a crear tablas

In [25]:
try:
    conexion= psycopg2.connect(
        database= "Teoriadia14",
        user= "postgres",
        password= "admin",
        host= "localhost",
        port= "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es erronea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrió el error {e}")

 

In [27]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS clientes(
    id_cliente SERIAL PRIMARY KEY,
    nombre VARCHAR(100),
    edad INT CHECK(edad >18)
);
"""
cursor.execute(query_creacion)   
conexion.commit()                   #El commit es para confirmar los cambios porque no estamos solo trayendo info sino haciendo cambios
query_insercion= "INSERT INTO clientes (nombre, edad) VALUES (%s, %s);"      #%s es como el format de los strings (la f)
valores= ("Lola", 19)    #Si queremos insertar un solo valor, hay que hacerlo con tupla
cursor.execute(query_insercion, valores)     #Hay que darle los valores que queremos insertar
conexion.commit()


valores_multiples= [               
    ("Marcos", 21),
    ("Mariano", 32)
    ]                             #Si queremos insertar varios valores con lista de tuplas

cursor.executemany(query_insercion, valores_multiples)                 #Executemany es para poder insertar varios valores
conexion.commit()



Vamos a ver como pasar un DataFrame a lista de tupla spara poder introducir sus valores a postrgres

In [28]:
df= pd.read_csv("datos\datos_ebay_zapatillas2024-10-15.csv", index_col=0)
df.head(2)

,nombre_zapatillas,imagen,precio
0,Adidas Hombres 7.5 NMD_V3 BOOST Zapatos Nube B...,https://i.ebayimg.com/thumbs/images/g/7YMAAOSw...,"169,48 EUR"
1,Adidas Pro Model Zapatos JH7147 Preloved Marró...,https://i.ebayimg.com/thumbs/images/g/X4IAAOSw...,"171,82 EUR"


In [34]:
tuple(df.values[0])    #Values da u array de listas y luego pasamos cada elemento a tupla con un for

lista_de_tuplas=[]
for fila in df.values:
    lista_de_tuplas.append(tuple(fila))
lista_de_tuplas2= [tuple(fila) for fila in df.values]    #El for se puede hacer en list comprehension
lista_de_tuplas[:3]

[('Adidas Hombres 7.5 NMD_V3 BOOST Zapatos Nube Blanco/Gris Dos HP9831 NUEVO',
  'https://i.ebayimg.com/thumbs/images/g/7YMAAOSwjfFnDgZF/s-l960.jpg',
  '169,48 EUR'),
 ('Adidas Pro Model Zapatos JH7147 Preloved Marrón/Núcleo Negro/Marfil',
  'https://i.ebayimg.com/thumbs/images/g/X4IAAOSwwxNnACBO/s-l960.jpg',
  '171,82 EUR'),
 ('Adidas Originals SL 72 RS Spark Core Negro Blanco IH7912 Hombres Talla',
  'https://i.ebayimg.com/thumbs/images/g/tJEAAOSwqSZm-ito/s-l960.jpg',
  '174,13 EUR')]

IMPORTANTE QUE LOS ID NO PODEMOS HACER SERIAL PORQUE ES UN LÍO ASI QUE HABRÁ QUE HACER UNA QUERY PARA CREAR LOS ID